# 쿼리의 구조

### SQL의 모습
```sql
SELECT * FROM table WHERE column="value"
```
### 몽고디비 쿼리의 모습
```python
collection.find({"field": "value"},{})
```
기본적을 쿼리는 필드가 가장 바깥에 있고, 안쪽에 연산자가 들어간다.<br>

하지만 예외적으로 **or, and, nor** 세 개의 연산자는 가장 바깥에 쓰인다.<br>
<br>
#### 정해진 형식을 지키지 않은 쿼리는 사용될 수 없다.

# 점 표기법
BSON 내부의 Object에 접근하기 위한 방법<br>

```python
{
    "_id":ObjectId("542c2b97 bac059 5474 108b48"),
    "name":{"first": "sue", "last": "Turing" },
    "age": 26,"is_alive":true,
    "groups": ["news", "sports"],
    "viewTime": ISODate("2017-10-24T05:02:46.395Z")
}
```

### name 안에 first를 조회하려면?

잘못된 쿼리 -> {"name" : {"first": 'sue'}}<br>
올바른 쿼리 -> {"name.first": 'sue}<br>

### 배열의 첫 번째 요소로 news값을 갖는 도큐먼트를 찾고 싶다면?

잘못된 쿼리 -> {"groups":['news']}<br>
올바른 쿼리 -> {"groups.0": "news"}

# 점표기법 실습
지시사항
1. name의 last 값이 Cate인 하나의 사람 도큐먼트에서 skills 필드를 제거하세요.
2. 수정한 도큐먼트를 pprint()로 출력하세요.
```python
import pymongo
from pprint import pprint
# 데이터베이스와 Collection을 생성하는 코드입니다. 수정하지 마세요!
connection = pymongo.MongoClient("mongodb://localhost:27017/")
db = connection.get_database("profile")
col = db.get_collection("people")
# name의 last 값이 "Cate"인 하나의 사람 도큐먼트 조건을 작성하세요.
query = {"name.last": "Cate"}
# 해당 도큐먼트를 수정하세요.
result = col.update_many(query,{"$unset": {"skills": False}})
# 수정한 도큐먼트를 조회하세요.
cursor = col.find(query)
# 앞선 결과를 출력하세요.
for people in cursor:
     pprint(people)


# 비교 연산자

|operator|설명|
|-|-|
| eq | (equals) 주어진 값과 일치하는 값|
| gt | (greater than) 주어진 값보다 큰 값|
| gte| (greather than or equals) 주어진 값보다 크거나 같은 값|
| lt| (less than) 주어진 값보다 작은 값|
| lte| (less than or equals) 주어진 값보다 작거나 같은 값|
| ne| (not equal) 주어진 값과 일치하지 않는 값|
| in|주어진 배열 안에 속하는 값|
| nin|주어진 배열 안에 속하지 않는 값|


## 비교예시
좋아요 수가 10 초과 30미만인 도큐먼트 검색
```python
articles.find({"likes":{"$gt":10, "$lt":30}})
```
좋아요 수가 10 이상 30이하인 도큐먼트 검색
```python
articles.find({"likes":{"$gte":10, "$lte":30}})
```
수량이 5 또는 15인 아이템 도큐먼트
```python
inventory.find( { "qty": { "$in": [ 5, 15]}})
```
태그가 정규표현식 ^be 또는 ^st에 일치하지않는 도큐먼트

```python
inventory.find( { "tags": { "$nin": [
    re.compile("^be"),
    re.compile("^st")
]} } )
```


# 비교연산자 실습 -조건에 맞지 않은 도큐먼트 삭제

지시사항<br>

1.date_received의 값은 예시와 같은 형식의 문자열로 저장되어 있습니다.

예시: "2012-04-01"<br>
2. date_received 값이 2015년도 혹은 2017년도가 아닌 책들을 모두 삭제하세요.
```python
import pymongo
from pprint import pprint
import re

# 데이터베이스와 Collection을 생성하는 코드입니다. 수정하지 마세요!
connection = pymongo.MongoClient("mongodb://localhost:27017/")
db = connection.get_database("library")
col = db.get_collection("books")

# date_received 값이 2015년도 혹은 2017년도가 아닌 조건을 작성하세요.
query = {"date_received": 
    {"$nin": [
        re.compile("^2015"),
        re.compile("^2017")
    ]}
}

# 조건에 맞는 도큐먼트를 삭제하세요.
col.delete_many(query)
```

# 비교연산자 실습 - 크기비교연산자

지시사항
1. date_received의 값은 다음과 같은 형식의 문자열로 저장되어 있습니다.
예시: "2012-04-01"<br>
2. date_received값이 2014년도부터 2017년도인 책들을 조회하세요.<br>

3. 그 책들의 '_id'와 'title' 필드만 도큐먼트 형식으로 for 문을 사용하여 pprint()로 출력해보세요

```python
import pymongo
from pprint import pprint
# 데이터베이스와 Collection을 생성하는 코드입니다. 수정하지 마세요!
connection = pymongo.MongoClient("mongodb://localhost:27017/")
db = connection.get_database("library")
col = db.get_collection("books")
# date_received 값이 2014년도부터 2017년도인 책들의 title 필드만 조회하세요.
query = {
    "date_received":{
        "$gte": "2014-01-01",
        "$lte": "2017-12-31"
    }
}
projection = {"_id":True, "title":True}
cursor = col.find(query, projection)
# books Collection에 들어있는 책들을 출력하세요.
for book in cursor:
    pprint(book)
```
